In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import erfc, erf
from scipy.integrate import quad
from collections import namedtuple
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.subplot.hspace'] = 0.25
plt.rcParams['figure.subplot.left'] = 0.17
plt.rcParams['axes.labelsize'] = 16

In [ ]:
# compute D, v_flow

fluid = namedtuple('fluid', ['tau', 'T', 'rho', 'alpha', 'm', 'a', 'eta'])
cell = namedtuple('cell', ['Ly', 'Lz', 'g'])

fluid.tau = 0.5
fluid.T = 0.33
fluid.rho = 10
fluid.alpha = 2.6
fluid.m = 1
fluid.a = 1

# Kapral review Eq. 55
eta_kin = fluid.T * fluid.tau * fluid.rho / (2*fluid.m) * \
    (5*fluid.rho-(fluid.rho - 1 + np.exp(-fluid.rho))*(2 - np.cos(fluid.alpha)-np.cos(2*fluid.alpha)))/ \
    ((fluid.rho - 1 + np.exp(-fluid.rho))*(2 - np.cos(fluid.alpha)-np.cos(2*fluid.alpha)))
# Kapral review Eq. 56
eta_coll = fluid.m / (18 * fluid.a * fluid.tau) * (fluid.rho - 1 + np.exp(-fluid.rho))*(1-np.cos(fluid.alpha))
fluid.eta = eta_kin + eta_coll
print("Viscosity", fluid.eta)
fluid.D = fluid.T*fluid.tau/(2*fluid.m) * (3*fluid.rho/((fluid.rho - 1 + np.exp(-fluid.rho))*(1-np.cos(fluid.alpha))) - 1)
print("Self-diffusion D", fluid.D)

cell.Ly = 32
cell.Lz = 15
cell.g = 1/1000

def v_of_eta(fluid, cell):
    return fluid.rho*cell.g*cell.Lz**2/(8*fluid.eta)

v_max = v_of_eta(fluid, cell)
v_av = 2/3*v_max
print("Flow maximum ", v_max)
print("Flow average ", v_av)
print("Poiseuille flow Peclet number", v_av*Lz/D_fluid)


In [ ]:
# define c_A(x,y) and lambda (derivative)

def c_A(x,y):
    return fluid.rho * 0.5*(1+erf(-(y-cell.Ly/2)/np.sqrt(4*fluid.D*x/v_max)))

def lam(x,y):
    return -fluid.rho*np.exp(-(y-cell.Ly/2)**2/(4*fluid.D*x/v_max))/np.sqrt(4*np.pi*fluid.D*x/v_max)

In [ ]:
X, Y = np.meshgrid(np.linspace(0.1, 20, 180),np.linspace(0, cell.Ly, 150))

plt.pcolormesh(X, Y, c_A(X, Y), cmap=plt.cm.viridis)
plt.colorbar()
plt.axis([X.min(), X.max(), 0, cell.Ly])


In [ ]:
plt.pcolormesh(X, Y, lam(X, Y), cmap=plt.cm.viridis)
plt.colorbar()
plt.axis([X.min(), X.max(), 0, cell.Ly])


In [ ]:
# define Lambda(R, eps)

def V(r, sigma, eps):
    return 4*eps*((sigma/r)**12-(sigma/r)**6) + eps

def integrand(r, sigma, eps):
    return r*np.exp(-V(r, sigma, eps)/fluid.T)

def Lambda(R, eps):
    result, error = quad(integrand, R/2, R, args=(sigma, eps))
    return result - R**2/2


In [ ]:
Lambda(3.1, 0.1)

## Single passive colloid

Here, the setup 

In [ ]:
# Single passive colloid
# Lambda lambda

sigma = 3
y_shift = 3.4

dt = 0.01
gamma = 4*np.pi*fluid.eta*sigma
D = fluid.T/gamma
x_factor = np.sqrt(2*D*dt)
y_factor = np.sqrt(2*D*dt)
F_factor = 8*np.pi*fluid.T/3 * R * (Lambda(R, 1)-Lambda(R, 0.25))/gamma

def run_single_passive():
    x, y = sigma, cell.Ly/2 + y_shift
    xy_data = []
    for t in range(1000):
        for tt in range(50):
            F_y = F_factor * lam(x, y)
            xi_x, xi_y = np.random.normal(size=(2,))
            x += v_max*dt + x_factor*xi_x
            y += F_y*dt + y_factor*xi_y
        xy_data.append((x,y))
    return np.array(xy_data)

single_passive = np.array([run_single_passive() for i in range(16)])

In [ ]:
m = single_passive.mean(axis=0).T
s = single_passive.std(axis=0).T
plt.plot(*m)
plt.plot(*(m+s))
plt.plot(*(m-s))

In [ ]:
fluid.eta

In [ ]:
# Single active colloid
# Lambda c_2

In [ ]:
sigma = 3
R = sigma*2**(1/6)
probability = 1
k0 = probability*R**2*np.sqrt(8*np.pi*fluid.T/fluid.m)
kD = 4*np.pi*R*fluid.D

print(k0, kD)


In [ ]:
v_max

In [ ]:
50/(1024*0.5)

In [ ]:
20/(500*0.5)

In [ ]:
plt.plot(xy_data[:,0])

In [ ]:
1/2/R*fluid.rho

In [ ]:
k0/(k0+kD)

In [ ]:
(0,)*10